In [12]:
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit

from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm

In [13]:
true_effect = 10
data = dowhy.datasets.linear_dataset(beta=true_effect,
        num_common_causes=5,
        num_instruments = 0,
        num_samples=10000,
        treatment_is_binary=True)
df = data["df"].copy()


In [14]:
df.head()

,W0,W1,W2,W3,W4,v0,y
0,2.278795,0.810755,0.557530,-1.394040,-1.586745,False,2.028816
1,0.573320,1.957450,-0.103575,2.062801,-0.785860,True,19.723423
2,1.271359,1.962679,1.463855,-0.087807,0.750286,True,22.862380
3,0.621021,1.851881,0.323006,0.199426,-2.025556,True,11.129416
4,1.548281,1.176890,-0.535305,1.478487,0.330723,True,22.426710


In [15]:
df[data['treatment_name'][0]] = [1 if x == True else 0 for x in df[data['treatment_name'][0]]]

In [11]:
df

,W0,W1,W2,W3,W4,v0,y,ps_0
0,1.565731,-1.843945,1.168437,1.359226,-1.574974,0,22.166579,0.593548
1,-0.426364,-0.793634,0.886457,0.468808,-2.654464,0,3.366061,0.045400
2,-2.620409,-0.423011,3.663157,0.859599,-2.319514,0,21.328450,0.999931
3,1.686354,-0.024532,0.217840,-0.744834,0.199222,0,4.683741,0.197728
4,1.373672,1.236214,1.144186,0.584845,-0.479613,0,21.663895,0.999964
...,...,...,...,...,...,...,...,...
9995,1.686422,0.465027,0.100017,0.928702,-0.551836,0,17.769921,0.994379
9996,0.021494,1.378701,2.729421,-0.519610,-1.277820,0,23.053569,0.999952
9997,-0.392347,0.851008,1.433023,1.733685,-0.601740,0,19.833649,0.999999
9998,1.286521,1.845992,-0.126820,-0.609454,-1.953795,0,2.924715,0.580998


In [16]:
logreg = LogisticRegression()
logreg.fit(df[data['common_causes_names']], df[data['treatment_name'][0]])
df['ps_0'] = logreg.predict_proba(df[data['common_causes_names']])[:,1]
df['ps_1'] = 1 - df['ps_0']

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(df[data['common_causes_names']], df[data['outcome_name']])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [ ]:
delta = 1.5

wt = (delta * df[data['treatment_name'][0]] + 1 - df[data['treatment_name'][0]]) / 
                                                 (delta * df[data['treatment_name'][0]] + 1 - df[data['treatment_name'][0]])
vt = (1 - delta) * (df[data['treatment_name'][0]] * (1 - df['ps_1']) - 
                    (1 - df[data['treatment_name'][0]]) * delta * df['ps_1']) / delta


df_pos = pd.concat([cov_outcome, treatment], axis = 1).drop('predicted_x', axis = 1).copy()
df_pos['x'] = 1
df_pos['treated_cf_outcome'] = rf.predict(df_pos)

df_neg = pd.concat([cov_outcome, treatment], axis = 1).drop('predicted_x', axis = 1).copy()
df_neg['x'] = 0
df_neg['control_cf_outcome'] = rf.predict(df_neg)

m0 = np.mean(df_pos['treated_cf_outcome'])
m1 = np.mean(df_neg['control_cf_outcome'])

rtp1 = (delta * treatment['predicted_x'] * m1 + (1 -  treatment['predicted_x']) * m0) / (delta *  treatment['predicted_x'] + 1 -  treatment['predicted_x'])



In [ ]:
wt*outcome['x'] + wt*vt*rtp1
